### Imports

In [10]:
import pandas as pd
import numpy as np
import csv

### Load Data

In [11]:
df = pd.read_csv('Data/data.csv',sep=";")
display(df.head())

,uri,date,author,claim,keywords,organization,rating
0,http://data.gesis.org/claimskg/creative_work/9...,03/26/2055,Yes on 55,California’s Prop 55 'prevents $4 billion in n...,"education,state finances",politifact,MIXTURE
1,http://data.gesis.org/claimskg/creative_work/e...,03/26/2020,Our Democracy 2020,Says an election clerk in Green Bay refused to...,"bipartisanship,city government,education,elect...",politifact,MIXTURE
2,http://data.gesis.org/claimskg/creative_work/d...,12/10/2019,America Today,'Queen Elizabeth removes Obamas from royal wed...,fake news,politifact,FALSE
3,http://data.gesis.org/claimskg/creative_work/0...,12/03/2019,One Wisconsin Now,Says state schools superintendent candidate Do...,"corporations,education,small business,state bu...",politifact,MIXTURE
4,http://data.gesis.org/claimskg/creative_work/f...,11/27/2019,One Wisconsin Now,"'The Walton family, which owns Wal-Mart, contr...","corporations,economy,income,labor,wealth",politifact,TRUE


### Get Clusters

In [12]:
clusters = [
    ['weather','oil spill','climate change','energy','environment'],
    ['pensions','state government spending','wealth','retirement','unions','labor','state finances','transportation','state budget','education','economy'],
    ['afghanistan','iraq','terrorism','military','foreign policy'],
    #['welfare','families','social security','deficit','federal budget','health care']
    ['weather','oil spill','climate change','energy','environment',
    'pensions','state government spending','wealth','retirement','unions','labor','state finances','transportation','state budget','education','economy',
    'afghanistan','iraq','terrorism','military','foreign policy']
]

for x in range(0, len(clusters)):
    cluster_index = 'c' + str(x+1)
    df[cluster_index] = False

In [13]:
#Add cluster info to DF
for index in range(0,len(df.index)):
    keywords = [x.strip() for x in df.at[index,'keywords'].split(',')]
    for x in range(0, len(clusters)):
        kwl = clusters[x]
        for keyword in keywords: 
            if keyword in kwl:
                cluster_index = 'c' + str(x+1)
                df.at[index,cluster_index] = True
                break

In [14]:
#Print statistics
for i in range (0, len(clusters)):
    print('c%d: %s' % (i+1,clusters[i]))

print()
    
for i in range (0, len(clusters)):
    x = 'c' + str(i+1)
    df2 = df.loc[df[x] == True]
    count = df2.shape[0]
    print('%s count: %d' % (x,count))

print()
    
for i in range(0, len(clusters)-1):
    for j in range(i+1, len(clusters)):
        x = 'c' + str(i+1)
        y = 'c' + str(j+1)
        df2 = df.loc[(df[x] == True) & (df[y] == True)]
        count = df2.shape[0]
        print('%s,%s overlap: %d' % (x,y,count))

print()
        
# df2 = df.loc[(df['c1'] == True) & (df['c2'] == True) & (df['c3'] == True)]
# count = df2.shape[0]
# print('c1,c2,c3 overlap: %d' % (count))

c1: ['weather', 'oil spill', 'climate change', 'energy', 'environment']
c2: ['pensions', 'state government spending', 'wealth', 'retirement', 'unions', 'labor', 'state finances', 'transportation', 'state budget', 'education', 'economy']
c3: ['afghanistan', 'iraq', 'terrorism', 'military', 'foreign policy']
c4: ['weather', 'oil spill', 'climate change', 'energy', 'environment', 'pensions', 'state government spending', 'wealth', 'retirement', 'unions', 'labor', 'state finances', 'transportation', 'state budget', 'education', 'economy', 'afghanistan', 'iraq', 'terrorism', 'military', 'foreign policy']

c1 count: 1037
c2 count: 4168
c3 count: 1441
c4 count: 6400

c1,c2 overlap: 122
c1,c3 overlap: 45
c1,c4 overlap: 1037
c2,c3 overlap: 84
c2,c4 overlap: 4168
c3,c4 overlap: 1441



### Save data

In [15]:
def save_df(df,filename):
    rows = list()
    header = ["id","text","author","date","keywords","organization","rating"]
    rows.append(header)
    
    for index,row in df.iterrows():
        claim = row.uri
        date = row.date
        author = row.author
        #keywords = row.keywords.replace(',','/')
        keywords = row.keywords
        orgName = row.organization
        ratingName = row.rating
        text = row.claim
        claim_row = [claim,text,author,date,keywords,orgName,ratingName]
        rows.append(claim_row)
        
    with open(filename,'w',newline='',encoding="utf-8") as writeFile:
        writer = csv.writer(writeFile,delimiter=',')
        writer.writerows(rows)
        writeFile.close()

In [16]:
#Individual Clusters
for i in range(0, len(clusters)):
    x = 'c' + str(i+1)
    df2 = df.loc[df[x] == True]
    filename = 'Data/' + x + '.csv'
    save_df(df2,filename)

#2 Clusters overlap
for i in range(0, len(clusters)-1):
    for j in range(i+1, len(clusters)):
        x = 'c' + str(i+1)
        y = 'c' + str(j+1)
        df2 = df.loc[(df[x] == True) & (df[y] == True)]
        filename = 'Data/' + x + '_' + y + '.csv'
        save_df(df2,filename)
        
#3 Clusters overlap
df2 = df.loc[(df['c1'] == True) & (df['c2'] == True) & (df['c3'] == True)]
save_df(df2,'Data/c1_c2_c3.csv')